<a href="https://colab.research.google.com/github/chaene82/loki_jupyter/blob/master/Tensorflow_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tensorflow learning
This is a notebook for learning Tensorflow

In [0]:
import tensorflow as tf
import pandas as pd

In [24]:
tf.__version__

'1.12.0'

### my first TensorFlow graph


In [25]:
a = tf.constant(5) 
b = tf.constant(2)
c = tf.constant(3)
d = tf.multiply(a,b) 
e = tf.add(c,b) 
f = tf.subtract(d,e) 
sess = tf.Session() 
outs = sess.run(f) 
sess.close() 
print("outs = {}".format(outs))

outs = 5


###Hello World

In [0]:
h = tf.constant("Hallo")

In [0]:
w = tf.constant(" Welt!")

In [0]:
hw = h + w


In [0]:
with tf.Session() as sess: 
  ans = sess.run(hw)


In [9]:
print(ans)

b'Hallo Welt!'


### MNIST - very simple

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
DATA_DIR = '/tmp/data'
NUM_STEPS = 1000 
MINIBATCH_SIZE = 100

In [12]:
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [0]:
x = tf.placeholder(tf.float32, [None, 784])

In [0]:
W = tf.Variable(tf.zeros([784, 10]))

In [0]:
y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)

In [0]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y_true))

In [0]:
gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [0]:
correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [0]:
with tf.Session() as sess: 
  # Train 
  sess.run(tf.global_variables_initializer()) 
  for _ in range(NUM_STEPS):

    batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
    sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})


  # Test 
  ans = sess.run(accuracy, feed_dict={x: data.test.images, y_true: 
  data.test.labels})



In [23]:
print("Genauigkeit: {:.4}%".format(ans*100))

Genauigkeit: 91.62%


### MNIST - CNN

In [0]:
import numpy as np

In [0]:
def weight_variable(shape): 
  initial = tf.truncated_normal(shape, stddev=0.1) 
  return tf.Variable(initial)



In [0]:
def bias_variable(shape): 
  initial = tf.constant(0.1, shape=shape) 
  return tf.Variable(initial)



In [0]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [0]:
def max_pool_2x2(x): 
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



In [0]:
def conv_layer(input, shape): 
  W = weight_variable(shape) 
  b = bias_variable([shape[3]]) 
  return tf.nn.relu(conv2d(input, W) + b)



In [0]:
def full_layer(input, size): 
  in_size = int(input.get_shape()[1]) 
  W = weight_variable([in_size, size]) 
  b = bias_variable([size]) 
  return tf.matmul(input, W) + b



In [0]:
STEPS = 5000

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [0]:
x_image = tf.reshape(x, [-1, 28, 28, 1]) 
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32]) 
conv1_pool = max_pool_2x2(conv1)



In [0]:
conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

In [0]:
conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])


In [0]:
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

In [0]:
keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full1_drop, 10)



In [87]:
mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv, labels=y_))


In [0]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [0]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



In [0]:
with tf.Session() as sess: 
  sess.run(tf.global_variables_initializer()) 
  for i in range(STEPS): 
    batch = mnist.train.next_batch(50) 
    if i % 100 == 0: 
      train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0}) 
      print("Schritt {}, Genauigkeit Anlernen {}".format(i, train_accuracy)) 
      
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5}) 
  X = mnist.test.images.reshape(10, 1000, 784) 
  Y = mnist.test.labels.reshape(10, 1000, 10) 
  
  test_accuracy = np.mean([sess.run(accuracy, feed_dict={x:X[i], y_:Y[i],keep_prob:1.0}) for i in range(10)])



Schritt 0, Genauigkeit Anlernen 0.10000000149011612
Schritt 100, Genauigkeit Anlernen 0.8799999952316284
Schritt 200, Genauigkeit Anlernen 0.8600000143051147
Schritt 300, Genauigkeit Anlernen 0.9200000166893005
Schritt 400, Genauigkeit Anlernen 0.9599999785423279
Schritt 500, Genauigkeit Anlernen 0.9399999976158142
Schritt 600, Genauigkeit Anlernen 1.0
Schritt 700, Genauigkeit Anlernen 0.9200000166893005
Schritt 800, Genauigkeit Anlernen 1.0
Schritt 900, Genauigkeit Anlernen 0.9200000166893005
Schritt 1000, Genauigkeit Anlernen 1.0
Schritt 1100, Genauigkeit Anlernen 0.9800000190734863
Schritt 1200, Genauigkeit Anlernen 0.9800000190734863
Schritt 1300, Genauigkeit Anlernen 0.9200000166893005
Schritt 1400, Genauigkeit Anlernen 0.9800000190734863
Schritt 1500, Genauigkeit Anlernen 0.9200000166893005
Schritt 1600, Genauigkeit Anlernen 0.9599999785423279
Schritt 1700, Genauigkeit Anlernen 0.9599999785423279
Schritt 1800, Genauigkeit Anlernen 1.0


In [0]:
print("Genauigkeit Test: {}".format(test_accuracy))


### CIFAR10